In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import sys, os
sys.path.append(os.path.abspath("..")) 
from data_scraping.db_utils import get_connection
from IPython.display import display

In [128]:
conn = get_connection()
cards_26_df = pd.read_sql("SELECT * FROM cards WHERE game = 26", conn)
cards_25_df = pd.read_sql("SELECT * FROM cards WHERE game = 25", conn)
sales_26_df = pd.read_sql("SELECT * FROM market_sales", conn)
prices_25_df = pd.read_sql("SELECT * FROM price_history", conn)

df_26 = sales_26_df.merge(cards_26_df, on='card_id', how='left')
df_25 = prices_25_df.merge(cards_25_df, on='card_id', how='left')


C:\Users\shane\AppData\Local\Temp\ipykernel_4304\441788012.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cards_26_df = pd.read_sql("SELECT * FROM cards WHERE game = 26", conn)
C:\Users\shane\AppData\Local\Temp\ipykernel_4304\441788012.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cards_25_df = pd.read_sql("SELECT * FROM cards WHERE game = 25", conn)
C:\Users\shane\AppData\Local\Temp\ipykernel_4304\441788012.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_26_df = pd.read_sql("SELECT * FROM market_sales"

### Cards Between FC 25 and FC 26 Similar Players Comparison

In [129]:
# Keep only name and price from last FIFA
date_target = '2024-09-23 23:59:59' 

fifa25_prices_df = pd.read_sql(f"""
    SELECT ph.card_id,
           COALESCE(ph.pc_value, ph.console_value) AS price
    FROM price_history ph
    JOIN (
        SELECT card_id, MAX(date_time) AS max_dt
        FROM price_history
        WHERE date_time <= '{date_target}'
          AND (pc_value > 0 OR console_value > 0)
        GROUP BY card_id
    ) sub
    ON ph.card_id = sub.card_id AND ph.date_time = sub.max_dt
""", conn)

# Merge card names
cards25_df = pd.read_sql("SELECT card_id, name FROM cards WHERE game=25", conn)
fifa25_prices_df = fifa25_prices_df.merge(cards25_df, on='card_id', how='left')

# --- Get last sold price per card in FIFA 26 ---
fifa26_last_price = (sales_26_df[sales_26_df['sold_price'] > 0]   # only valid sales
                     .sort_values(['card_id','sale_time'])
                     .groupby('card_id')
                     .agg(last_price=('sold_price','last'))   # last sold price
                     .reset_index())

# Merge card names
cards26_df = pd.read_sql("SELECT card_id, name FROM cards WHERE game=26", conn)
fifa26_last_price = fifa26_last_price.merge(cards26_df, on='card_id', how='left')

# Normalize names for merging
fifa25_prices_df['name'] = fifa25_prices_df['name'].str.lower().str.strip()
fifa26_last_price['name'] = fifa26_last_price['name'].str.lower().str.strip()

# Merge FIFA 25 and FIFA 26 prices on name
price_comparison = fifa25_prices_df.merge(
    fifa26_last_price[['name','last_price']],
    on='name',
    how='inner'
)

# Compute % difference
price_comparison['pct_diff'] = ((price_comparison['last_price'] - price_comparison['price']) 
                                / price_comparison['price']) * 100

# Filter similar prices ±10%
similar_prices = price_comparison[(price_comparison['pct_diff'].abs() <= 20) & (price_comparison['last_price'] > 5000) & (price_comparison['last_price'] < 30000)].sort_values('pct_diff', ascending=False)



print(similar_prices[['name','price','last_price','pct_diff']])


C:\Users\shane\AppData\Local\Temp\ipykernel_4304\982070285.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fifa25_prices_df = pd.read_sql(f"""
C:\Users\shane\AppData\Local\Temp\ipykernel_4304\982070285.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cards25_df = pd.read_sql("SELECT card_id, name FROM cards WHERE game=25", conn)


                               name  price  last_price   pct_diff
956                    jonathan tah   6015        7000  16.375727
588                       ewa pajor  10969       12750  16.236667
132             alexis mac allister   9107       10500  15.295926
392                     luka modric   8308        9500  14.347617
133             alexis mac allister   9269       10500  13.280829
112                  victor osimhen  12610       14250  13.005551
393                     luka modric   8923        9500   6.466435
589                       ewa pajor  11977       12750   6.454037
125               emiliano martinez  13915       14500   4.204096
461           debora c. de oliveira  27635       28750   4.034739
328                     chloe kelly   8576        8900   3.777985
214                    paulo dybala  25578       26000   1.649855
681                  josko gvardiol   7682        7800   1.536058
866                     yann sommer   9911       10000   0.897992
1067      

C:\Users\shane\AppData\Local\Temp\ipykernel_4304\982070285.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cards26_df = pd.read_sql("SELECT card_id, name FROM cards WHERE game=26", conn)


#### Meta Main League CB Prices Comparison (Over 80 Pace and 80 Defending)

In [130]:
# Load stats tables
pace_df = pd.read_sql("SELECT * FROM card_pace_stats", conn)
shooting_df = pd.read_sql("SELECT * FROM card_shooting_stats", conn)
passing_df = pd.read_sql("SELECT * FROM card_passing_stats", conn)
dribbling_df = pd.read_sql("SELECT * FROM card_dribbling_stats", conn)
defending_df = pd.read_sql("SELECT * FROM card_defending_stats", conn)
physical_df = pd.read_sql("SELECT * FROM card_physical_stats", conn)

pace_df = pace_df.drop(columns=['id'], errors='ignore')
shooting_df = shooting_df.drop(columns=['id'], errors='ignore')
passing_df = passing_df.drop(columns=['id'], errors='ignore')
dribbling_df = dribbling_df.drop(columns=['id'], errors='ignore')
defending_df = defending_df.drop(columns=['id'], errors='ignore')
physical_df = physical_df.drop(columns=['id'], errors='ignore')

date_target = '2024-09-23 23:59:59' 

# Merge all stats with cards info
full_stats_df = df_25.merge(pace_df, on="card_id", how="left") \
                        .merge(shooting_df, on="card_id", how="left") \
                        .merge(passing_df, on="card_id", how="left") \
                        .merge(dribbling_df, on="card_id", how="left") \
                        .merge(defending_df, on="card_id", how="left") \
                        .merge(physical_df, on="card_id", how="left")

main_league_cb_26 = full_stats_df[
    (
        (full_stats_df["league"] == "Ligue 1 Uber Eats")
        | (full_stats_df["league"] == "Premier League")
        | (full_stats_df["league"] == "LALIGA EA SPORTS")
        | (full_stats_df["league"] == "Serie A TIM")
        | (full_stats_df["league"] == "Bundesliga")
    )
    & (full_stats_df["position"] == "CB")
    & (full_stats_df["pace_overall"] >= 80)
    & (full_stats_df["defending_overall"] >= 82)
    & (full_stats_df["date_time"] <= date_target)
    & (full_stats_df["pc_value"] >= 1000)
    & (full_stats_df["pc_value"] <= 40000)
]

print(main_league_cb_26)



C:\Users\shane\AppData\Local\Temp\ipykernel_4304\589323997.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pace_df = pd.read_sql("SELECT * FROM card_pace_stats", conn)
C:\Users\shane\AppData\Local\Temp\ipykernel_4304\589323997.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  shooting_df = pd.read_sql("SELECT * FROM card_shooting_stats", conn)
C:\Users\shane\AppData\Local\Temp\ipykernel_4304\589323997.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  passing_df = pd.read_sql("SELECT * FROM card_passing_stats", conn)

        card_id  pc_value  console_value           date_time  \
441723    33134   28493.0            NaN 2024-09-23 09:30:00   
594563    43991   38942.0            NaN 2024-09-23 09:30:00   

                                   name  game    version nationality  \
441723  Gleison Bremer Silva Nascimento  25.0  Gold Rare      Brazil   
594563                    Fikayo Tomori  25.0  Gold Rare     England   

             league       club  ... interceptions  heading_acc  def_aware  \
441723  Serie A TIM   Juventus  ...            84           87         87   
594563  Serie A TIM  Milano FC  ...            84           83         84   

        stand_tackle  slide_tackle physical_overall  jumping  stamina  \
441723            87            84               85       94       75   
594563            87            81               80       92       71   

        strength  aggression  
441723        89          84  
594563        81          88  

[2 rows x 51 columns]


In [138]:
# Filter recent data (last 24h)
recent_df = df_26[(df_26['sale_time'] > df_26['sale_time'].max() - pd.Timedelta(hours=24)) 
                  & (df_26['sold_price'] > 6000) 
                  & (df_26['platform'] == "pc")]

buy_candidates = []

# Loop through each card
for name, group in recent_df.groupby('name'):
    group = group.sort_values('sale_time', ascending=False)
    
    if len(group) >= 50:  # need enough sales for comparison
        last_short_avg = group.head(10)['sold_price'].mean()  # short-term average
        last_long_avg = group.head(60)['sold_price'].mean()   # longer-term average

        sales_volume = group.shape[0]

        # Check drop & liquidity
        if last_short_avg < 0.9 * last_long_avg and sales_volume >= 20:
            buy_price = round(last_short_avg * 0.97)   # buy slightly below dip
            sell_price = round(last_long_avg * 0.98)  # sell slightly below average

            buy_candidates.append({
                "name": name,
                "last_10_avg": round(last_short_avg, 2),
                "last_50_avg": round(last_long_avg, 2),
                "drop_%": round((last_long_avg - last_short_avg) / last_long_avg * 100, 2),
                "sales_volume": sales_volume,
                "suggested_buy": buy_price,
                "suggested_sell": sell_price,
                "potential_profit": sell_price - buy_price
            })

# Convert to DataFrame
buy_df = pd.DataFrame(buy_candidates)

if not buy_df.empty:
    buy_df = buy_df.sort_values("drop_%", ascending=False)
    print("🔥 Potential Buy Candidates 🔥")
    print(buy_df.head(20))
else:
    print("No buy candidates found with current filters.")


🔥 Potential Buy Candidates 🔥
           name  last_10_avg  last_50_avg  drop_%  sales_volume  \
1   Lauren Hemp      12350.0     14016.67   11.89           424   
0  Felix Nmecha       7860.0      8795.83   10.64           407   

   suggested_buy  suggested_sell  potential_profit  
1          11980           13736              1756  
0           7624            8620               996  


In [135]:
# Filter last 24h of sales
recent_df = df_26[(df_26['sale_time'] > df_26['sale_time'].max() - pd.Timedelta(hours=24)) 
                  & (df_26['sold_price'] > 6000) 
                  & (df_26['platform'] == "pc")]

volatility_candidates = []

# Loop through each card
for name, group in recent_df.groupby('name'):
    group = group.sort_values('sale_time', ascending=False)
    
    if len(group) >= 200:  # need enough sales to judge stability
        last_200 = group.head(200)['sold_price']
        avg_price = last_200.mean()
        std_dev = last_200.std()
        cv = std_dev / avg_price  # coefficient of variation (volatility measure)

        # Look for stability (low volatility, e.g. <5%) 
        # and presence of dips (at least one sale <90% of avg)
        if cv < 0.05 and (last_200.min() < 0.9 * avg_price):
            volatility_candidates.append({
                "name": name,
                "avg_price": round(avg_price, 2),
                "std_dev": round(std_dev, 2),
                "volatility_%": round(cv * 100, 2),
                "lowest_sale": last_200.min(),
                "undercut_%": round((avg_price - last_200.min()) / avg_price * 100, 2),
                "sales_volume": len(last_200)
            })

# Convert to DataFrame
vol_df = pd.DataFrame(volatility_candidates).sort_values("undercut_%", ascending=False)

print("📉 Stable Cards with Undercuts (Low Volatility Snipes) 📉")
print(vol_df.head(20))

📉 Stable Cards with Undercuts (Low Volatility Snipes) 📉
                                 name  avg_price  std_dev  volatility_%  \
11                      Fikayo Tomori    8783.50   320.77          3.65   
8                         Declan Rice   17331.25   863.46          4.98   
1                   Antoine Griezmann   17408.75   782.65          4.50   
5                         Bukayo Saka   18966.25   800.71          4.22   
6               David De Gea Quintana   12407.00   471.71          3.80   
4                     Bradley Barcola    9089.00   417.19          4.59   
2                     Antonio Rudiger   71162.50  2967.55          4.17   
27              Rodrygo Silva de Goes   65027.50  2557.97          3.93   
3                 Aurelien Tchouameni   15645.00   675.93          4.32   
14                      Heung Min Son   12696.25   519.64          4.09   
31                      Sandro Tonali   21675.00   889.64          4.10   
7                     Dayot Upamecano   1895

In [133]:
vol_df['strategy'] = 'Low-Volatility Snipe'
buy_df['strategy'] = 'Momentum Crash'

# --- Merge both datasets ---
buy_list = pd.concat([vol_df, buy_df], ignore_index=True)

# --- Optional: filter by key metrics ---
# Example filters: volatility < 5% and undercut > 10%
buy_list_filtered = buy_list[
    (buy_list['volatility_%'] < 5) & 
    (buy_list['undercut_%'] > 10)
]

# --- Sort by potential profit (undercut % descending) ---
buy_list_sorted = buy_list_filtered.sort_values(by='undercut_%', ascending=False)

# --- Save the combined BUY LIST ---
buy_list_sorted.to_csv("combined_buy_list.csv", index=False)

print("✅ Combined BUY LIST generated!")
print(buy_list_sorted.head(20))  # Preview top 20 cards

✅ Combined BUY LIST generated!
                         name  avg_price  std_dev  volatility_%  lowest_sale  \
0                Dean Huijsen     787.00    34.81          4.42        600.0   
1               Fikayo Tomori    8783.50   320.77          3.65       6700.0   
2                 Declan Rice   17331.25   863.46          4.98      13250.0   
3           Antoine Griezmann   17408.75   782.65          4.50      13500.0   
4                 Bukayo Saka   18966.25   800.71          4.22      15000.0   
5       David De Gea Quintana   12407.00   471.71          3.80       9900.0   
6             Bradley Barcola    9089.00   417.19          4.59       7300.0   
7   Gabriel Fernando de Jesus     867.00    38.32          4.42        700.0   
8             Antonio Rudiger   71162.50  2967.55          4.17      57500.0   
9         Aurelien Tchouameni   15645.00   675.93          4.32      12750.0   
10      Rodrygo Silva de Goes   65027.50  2557.97          3.93      53000.0   
11       